In [1]:
import mat73
import argparse
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

from biomarkers import BioMarkers, EEG, EMG, EOG
from feature_extraction import concatenate_features
from data_utils import load_data_from_dir

In [2]:
DIR_NAME = "../1000_CleanData"

all_data = load_data_from_dir(DIR_NAME)

Complete loading 10 markers
Loaded audio_nvla block
Complete loading 10 markers
Loaded breath_hvha block
Complete loading 10 markers
Loaded mind wandering block
Complete loading 10 markers
Loaded breath_nvha block
Complete loading 10 markers
Loaded audio_hvla block
Complete loading 10 markers
Loaded breath_nvla block
Complete loading 10 markers
Loaded meditation block
Complete loading 10 markers
Loaded audio_hvha block
Complete loading 10 markers
Loaded breath_hvla block


In [8]:
# valance labels
all_labels = np.array([])
for block, markers in all_data.items():
    y = markers.get_labels()
    print(f'{block} block valence labels {y}')
    all_labels = np.concatenate((all_labels, y), axis=None)

print(all_labels.shape)

audio_nvla block valence labels [0.75875 0.59125 0.6075  0.52125 0.42875 0.665   0.49125 0.57375 0.34
 0.5725  0.355   0.43375 0.48375]
breath_hvha block valence labels [0.77125 0.695   0.64375 0.70125 0.71125 0.6225  0.6225  0.72375 0.65125
 0.68625 0.75    0.61625 0.58875]
mind wandering block valence labels [0.5     0.47    0.445   0.40625 0.46375 0.39    0.75875 0.59125 0.6075
 0.52125 0.42875 0.665   0.49125]
breath_nvha block valence labels [0.51875 0.4175  0.595   0.5075  0.65125 0.5475  0.37375 0.3225  0.605
 0.20625 0.58375 0.6175  0.5825 ]
audio_hvla block valence labels [0.92625 0.9525  0.89625 0.96875 0.9375  0.91875 0.9425  0.8375  0.78625
 0.865   0.6175  0.90625 0.71125]
breath_nvla block valence labels [0.5     0.60375 0.665   0.54375 0.58125 0.405   0.615   0.54375 0.5375
 0.5825  0.5675  0.5725  0.42   ]
meditation block valence labels [0.5     0.5675  0.5     0.68625 0.85625 0.48    0.56875 0.3925  0.54
 0.59125 0.15625 0.41625 0.77125]
audio_hvha block valence label

In [11]:
from data_utils import get_features_in_all_blocks

all_features = get_features_in_all_blocks(all_blocks=all_data)

ImportError: cannot import name 'get_features_in_all_blocks' from 'data_utils' (/Users/tpan/Development/volunteer/MM_UCSF/analysis/data_utils.py)

In [26]:
# Train machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate

In [24]:
groups_list=[[i] for i, j in enumerate(y)]
group_array=np.hstack(groups_list)

print(group_array)
print(all_features.shape, y.shape, group_array.shape)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
(13, 1518) (13,) (13,)


In [28]:
#convert y values to categorical values
lab = LabelEncoder()
y_transformed = lab.fit_transform(y)

#view transformed values
print(y_transformed)
print(y)

[ 6  2  0  9  7 11 10  4  3  1  2  5  8]
[0.89625 0.825   0.80375 0.9425  0.9075  0.95375 0.9525  0.8875  0.87125
 0.815   0.825   0.89125 0.9275 ]


In [33]:
clf=LogisticRegression(max_iter=200)
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=16)
gscv.fit(all_features,y_transformed,groups=group_array)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        LogisticRegression(max_iter=200))]),
             n_jobs=16,
             param_grid={'classifier__C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1,
                                           10, 100]})

In [34]:
gscv.best_score_
# importance = clf.coef_[0]
# #importance is a list so you can plot it.
# feat_importances = pd.Series(importance)
# feat_importances.nlargest(20).plot(kind='barh',title = 'Feature Importance')

0.0

/Users/tpan/miniconda3/envs/MM/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tpan/miniconda3/envs/MM/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

/Users/tpan/miniconda3/envs/MM/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tpan/miniconda3/envs/MM/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht